### Preprocess

In [68]:
# import model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from PIL import Image
#from sklearn.metrics import f1_score
#from matplotlib.animation import FuncAnimation
#from google.colab import drive
#drive.mount('/content/drive')

Read Data

In [69]:
def _preprocess(image_arr, th=0.4):
        image_arr[image_arr > th] = 1.0
        image_arr[image_arr <= th] = 0.0
        return image_arr

In [ ]:
def SplitData(data):

    X_train = data.iloc[:20000, :3].values  # Get training features
    X_val = data.iloc[20000:, :3].values    # Get validation features
    y_train = data.iloc[:20000, 3].values   # Get training labels
    y_val = data.iloc[20000:, 3].values      # Get validation labels

    return X_train, X_val, y_train, y_val
X_train, X_val, y_train, y_val = SplitData(train)
X_test = test.iloc[:, :3].values

In [ ]:
from PIL import Image


def image_resize(path):
    # path : the image path you want to resize
    # img_resized: img have resize to 360*360
    #open picture
    img = Image.open("D:\MachineLearning_FinalProject\code\photo\JayChou.jpg")
    # resize to 360*360
    img_resized = img.resize((360, 360), Image.ANTIALIAS)
    return img_resized

In [80]:
def read_images_from_folder(folder_path):
    # read images from 'folder_path'
    # flat the image and push into 'images' array
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.jpg'):
            file_path = os.path.join(folder_path, filename)
            try:
                # 打開圖片
                with Image.open(file_path) as img:
                    # 將圖片轉為灰階
                    img = img.convert('L')
                    # 檢查圖片尺寸是否為 360x360
                    if img.size == (190, 195):
                        # 將圖片轉換為 np.array 並進行標準化 (值範圍從 0 到 1)
                        img_array = np.array(img) / 255.0
                        # 將圖片展平 (flatten)
                        img_array = img_array.flatten()
                        images.append(img_array)
                        print(images[0])
                        print(f'image size: {len(images[0])}')
                    else:
                        print(f"圖片尺寸為 {img.size}, 不是 360x360: {filename}")
            except UnidentifiedImageError:
                print(f"無法識別圖片文件: {filename}")
            except Exception as e:
                print(f"無法讀取圖片: {filename}, 錯誤: {e}")
    
    return images

In [81]:
folder_path = "D:\MachineLearning_FinalProject\code\photo\\"
preprocess_photo = read_images_from_folder(folder_path)
for i in range(len(preprocess_photo)):
    preprocess_photo[i] = _preprocess(preprocess_photo[i])
#preprocess_photo


NameError: name 'UnidentifiedImageError' is not defined

### Model

In [ ]:
# neural network
# CNN
# SVM
def Model():
    pass

In [ ]:
class NN(): 
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, activation):   
        self.W1 = np.random.randn(input_size, hidden_size1) / np.sqrt(input_size);
        self.b1 = np.zeros([1, hidden_size1]);
        self.W2 = np.random.randn(hidden_size1, hidden_size2) / np.sqrt(input_size);
        self.b2 = np.zeros([1, hidden_size2]);
        self.W3 = np.random.randn(hidden_size2, output_size) / np.sqrt(input_size);
        self.b3 = np.zeros([1, output_size]);
        
        self.placeholder = {"x":None, "y":None};
        self.activation = activation;
        # pass
    
    # Feed Placeholder
    def feed(self, feed_dict):
        for key in feed_dict :
            self.placeholder[key] = feed_dict[key].copy();
        # pass
    
    # Forward Propagation
    def forward(self):
        n = self.placeholder["x"].shape[0];
        self.A1 = np.ones((n, 1)).dot(self.b1) + self.placeholder["x"].dot(self.W1);
        self.H1 = np.maximum(self.A1, 0);
        self.A2 = np.ones((n, 1)).dot(self.b2) + self.H1.dot(self.W2);
        self.H2 = np.maximum(self.A2, 0);
        self.A3 = np.ones((n, 1)).dot(self.b3) + self.H2.dot(self.W3);
        
        if self.activation == "linear" :
            self.y = self.A3.copy();
        elif self.activation == "sigmoid" :
            self.y = 1.0 / (1.0 + np.exp(-self.A3));
        elif self.activation == "softmax" :
            self.logit_y = np.exp(self.A3 - np.max(self.A3, 1, keepdims=True));
            self.y = self.logit_y / np.sum(self.logit_y, 1, keepdims=True);
        
        return self.y;
        # pass
    
    # Backward Propagation
    def backward(self):
        n = self.placeholder["y"].shape[0];
        self.grab_A3 = (self.y - self.placeholder["y"]) / n;
        self.grab_b3 = np.ones((n, 1)).T.dot(self.grab_A3);
        self.grab_W3 = self.H2.T.dot(self.grab_A3);
        self.grab_H2 = self.grab_A3.dot(self.W3.T);
        self.grab_A2 = self.grab_H2 * (self.A2 > 0).astype(float);
        self.grab_b2 = np.ones((n, 1)).T.dot(self.grab_A2);
        self.grab_W2 = self.H1.T.dot(self.grab_A2);
        self.grab_H1 = self.grab_A2.dot(self.W2.T);
        self.grab_A1 = self.grab_H1 * (self.A1 > 0).astype(float);
        self.grab_b1 = np.ones((n, 1)).T.dot(self.grab_A1);
        self.grab_W1 = self.placeholder["x"].T.dot(self.grab_A1); 
        # pass
    
    # Update Weights
    def update(self, learning_rate=1e-3):
        self.W3 = self.W3 - learning_rate*self.grab_W3;
        self.W2 = self.W2 - learning_rate*self.grab_W2;
        self.W1 = self.W1 - learning_rate*self.grab_W1;
        self.b3 = self.b3 - learning_rate*self.grab_b3;
        self.b2 = self.b2 - learning_rate*self.grab_b2;
        self.b1 = self.b1 - learning_rate*self.grab_b1;
        # pass
    
    # Loss Functions
    def computeLoss(self):
        loss = 0.0;
        if(self.activation == "linear") : 
            loss = 0.5 * np.square(self.y - self.placeholder["y"]).mean();
        elif(self.activation == "softmax") :
            loss = np.sum((-self.placeholder["y"] * np.log(self.y + 1e-6)) , 1).mean();
        elif(self.activation == "sigmoid") :
            loss = np.sum((-self.placeholder["y"] * np.log(self.y + 1e-6)) 
                          - (1-self.placeholder["y"]) * np.log(1-self.y + 1e-6)).mean();
        return loss;
        pass